In [ ]:
import keras_tuner as kt
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os


In [ ]:
# 加载数据集中的Qd、循环寿命和总容量信息，并进行分析
temp_Qd = np.load("Data_processed/Qd_100.npy", allow_pickle=True).item()
temp_life = np.load("Data_processed/cycle_life.npy", allow_pickle=True).item()
all_capacity = np.load("Data_processed/all_capacity.npy", allow_pickle=True).item()


In [3]:

temp_Qd_all = []
temp_life_all = []

for key in temp_Qd.keys():
    temp_life_all.append(temp_life[key])
    temp_Qd_list = []
    for item in temp_Qd[key]: 
        temp_Qd_list.append(item)
    temp_Qd_arr = np.asarray(temp_Qd_list)
    temp_Qd_all.append(temp_Qd_arr)

all_Qd_arr = np.asarray(temp_Qd_all)
cycle_life_arr = np.asarray(temp_life_all)

In [4]:
test_ind = np.hstack((np.arange(0, (41 + 43), 2), 83))
train_ind = np.arange(1, (41 + 43 - 1), 2)
secondary_true_ind = np.arange(124 - 40, 124)

all_keys = list(temp_Qd.keys())
train_keys = [all_keys[inx] for inx in train_ind]
test_keys = [all_keys[inx] for inx in test_ind]
secondary_true_keys = [all_keys[inx] for inx in secondary_true_ind]

cycle_life_arr = np.asarray(cycle_life_arr).reshape(-1, 1)
max_label = np.max(cycle_life_arr)
cycle_life_arr = cycle_life_arr / max_label

train_Qds = np.asarray(all_Qd_arr)[train_ind]
train_cycle_lifes = np.asarray(cycle_life_arr)[train_ind]

# 测试集，用于最后测试模型啮合程度
test_Qd_a = np.asarray(all_Qd_arr)[test_ind]
test_cycle_life_a = np.asarray(cycle_life_arr)[test_ind]

test_Qd_b = np.asarray(all_Qd_arr)[secondary_true_ind]
test_cycle_life_b = np.asarray(cycle_life_arr)[secondary_true_ind]


In [5]:
# 分解数据集，训练集和验证集
train_Qd, val_Qd, train_cycle_life, val_cycle_life = train_test_split(train_Qds, train_cycle_lifes, test_size=0.2, random_state=42)

In [6]:
train_Qd = torch.FloatTensor(train_Qd) 
val_Qd = torch.FloatTensor(val_Qd)
train_cycle_life = torch.FloatTensor(train_cycle_life).view(-1, 1)
val_cycle_life = torch.FloatTensor(val_cycle_life).view(-1, 1)

# 增加一个维度以匹配模型的输入要求
test_Qd_a = torch.FloatTensor(test_Qd_a)
test_cycle_life_a = torch.FloatTensor(test_cycle_life_a).view(-1, 1)

test_Qd_b = torch.FloatTensor(test_Qd_b)
test_cycle_life_b = torch.FloatTensor(test_cycle_life_b).view(-1, 1)

# # 增加一个维度以匹配模型的输入要求
# train_Qd = train_Qd.unsqueeze(1) 
# val_Qd = val_Qd.unsqueeze(1)
# test_Qd_a = test_Qd_a.unsqueeze(1)
# test_Qd_b = test_Qd_b.unsqueeze(1)

#预处理数据，normalize
train_Qd_mean = train_Qd.mean()
train_Qd_std = train_Qd.std()

train_Qd = (train_Qd - train_Qd_mean) / train_Qd_std
val_Qd = (val_Qd - train_Qd_mean) / train_Qd_std
test_Qd_a = (test_Qd_a - train_Qd_mean) / train_Qd_std
test_Qd_b = (test_Qd_b - train_Qd_mean) / train_Qd_std



In [7]:
#keras tuner调试cnn-lstm模型
# 定义模型
def build_model(hp):
    model = Sequential()            
    model.add(Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32),
                     kernel_size=hp.Choice('kernel_size', values=[3, 5, 7]),
                     activation='relu',
                     input_shape=(train_Qd.shape[1]-1, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   activation='tanh',
                   return_sequences=True))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    return model


In [ ]:
# 文件目录不支持中文路径

tuner = kt.RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=5,
        overwrite=True,
        executions_per_trial=4,
        directory='my_dir',
        project_name='my_rnd_search',
        seed=42 
    )

# 设置早停回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5)


# 执行超参数搜索

tuner.search(
        train_Qd,
        train_cycle_life,
        epochs=10,
        # batch_size=32,
        validation_data=(val_Qd, val_cycle_life),
        callbacks=[early_stopping]
    )





In [ ]:

# 获取最佳超参数
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # 重新构建模型
model = tuner.hypermodel.build(best_hps)

# 验证模型
model.eval()
criterion = nn.MSELoss()

with torch.no_grad():
    predictions = model(val_Qd)
    val_loss = criterion(predictions, val_cycle_life)
    print(f'Validation Loss: {val_loss.item()}')

# 测试模型
with torch.no_grad():
    predicted_a = model(test_Qd_a)
    predicted_b = model(test_Qd_b)
    test_loss_a = criterion(predicted_a, test_cycle_life_a)
    test_loss_b = criterion(predicted_b, test_cycle_life_b)
    print(f'Test Loss A: {test_loss_a.item()}')
    print(f'Test Loss B: {test_loss_b.item()}')


In [ ]:
    
# 绘制预测值与真实值的对比图
import matplotlib.pyplot as plt
test_cycle_life=[test_cycle_life_a,test_cycle_life_b]
for cycle_life in test_cycle_life:
    pred_cycle_life=model.predict(cycle_life)
    plt.plot(cycle_life, label='Actual')
    plt.plot(pred_cycle_life, label='Predicted')
    plt.legend()
    plt.show()
# 保存模型
model.save('battery_life_model.h5')
